In [1]:
import os
import re
import sys
import json
import fnmatch
import spotipy
import pandas as pd
import sqlite3
from sqlite3 import Error
from tqdm import tqdm
from zipfile import ZipFile
from datetime import datetime
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [2]:
cwd = os.getcwd()
sys.path.insert(1, os.path.join(cwd, '..'))
import config
# Spotify credentials
os.environ["SPOTIPY_CLIENT_ID"] = config.SPOTIPY_CLIENT_ID
os.environ["SPOTIPY_CLIENT_SECRET"] = config.SPOTIPY_CLIENT_SECRET
os.environ['SPOTIPY_REDIRECT_URI'] = config.SPOTIPY_REDIRECT_URI

In [ ]:
def loop_slices(path, num_slices=20):
    cnt = 0
    mpd_playlists = []
    filenames = os.listdir(path)

    for fname in sorted(filenames):
        print(fname)
        if fname.startswith("mpd.slice.") and fname.endswith(".json"):
            cnt += 1
            fullpath = os.sep.join((path, fname))
            f = open(fullpath)
            js = f.read()
            f.close()
            current_slice = json.loads(js)

            # Create a list of all playlists
            for playlist in current_slice['playlists']:
                mpd_playlists.append(playlist)

            if cnt == num_slices:
                break
    return mpd_playlists

# Path where the json files are extracted
path = 'data/'
playlists = loop_slices(path, num_slices=20)

In [ ]:
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

cols_to_keep = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

dfs = []
for playlist in tqdm(playlists):
   audio_feats = []
   for track in playlist['tracks']:
      track_uri = track['track_uri'].split(":")[2]
      feature = sp.audio_features(track_uri)
      if feature:
         audio_feats.append(feature[0])
   avg_feats = pd.DataFrame(audio_feats)[cols_to_keep].mean()
   avg_feats['name'] = playlist['name']
   avg_feats['pid'] = playlist['pid']
   dfs.append(avg_feats.T)

In [ ]:
def loop_slices_zip(zip_file, num_slices=20):
    #print("Number of processors: ", mp.cpu_count())
    with ZipFile(zip_file) as zipfiles:
        file_list = zipfiles.namelist()

        #get only the csv files
        json_files = fnmatch.filter(file_list, "*.json")
        json_files_ex = fnmatch.filter(file_list, "*MACOSX*")
        json_files = set(json_files) - set(json_files_ex)
        json_files = [f for i,f in sorted([(int(fn.split('.')[2].split('-')[0]), fn) for fn in json_files])]
        print(json_files[:5])

        cnt = 0
        mpd_playlists = []
        for filename in tqdm(json_files):
            cnt+=1
            with zipfiles.open(filename) as json_file:
                current_slice = json.loads(json_file.read())
                
                # Create a list of all playlists
                for playlist in current_slice['playlists']:
                    mpd_playlists.append(playlist)

                if cnt == num_slices:
                    break
        return mpd_playlists

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

cols_to_keep = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
dfs = []

for playlist in tqdm(playlists):
    audio_feats = []
    all_uris_in_plylst = []
    for track in playlist['tracks']:
        uri = track['track_uri'].split("k:")[1]
        all_uris_in_plylst.append(uri)
    #print('pid : ', playlist['pid'])

    chunks_uris = [all_uris_in_plylst[i:i + 100] for i in range(0, len(all_uris_in_plylst), 100)]
    for chunk in  chunks_uris:
        for attempt in range(10):
            try:
                chunk_audio_feats = sp.audio_features(chunk)
                audio_feats.append(chunk_audio_feats)
            except Exception as e: 
                print(e)
                print('playlist: {}, chunk: {}'.format(playlist['name'], chunk))
            else:
                break
        else:
            print('Everything failed')
    
    playlist_audio_feats = [item for sublist in audio_feats for item in sublist]
    name = playlist['name']
    pid = playlist['pid']
    s1 = pd.Series([name, pid], index=['name', 'pid'])
    try: # Try/Except for when there is a None in playlist
        s2 = pd.DataFrame(playlist_audio_feats)[cols_to_keep].mean()
    except:
        print('Playlist "{}" has a None. PID: {}'.format(name, pid))
        s2 = pd.DataFrame([i for i in playlist_audio_feats if i])[cols_to_keep].mean() # List comprehension to remove None from list

    dfs.append(pd.DataFrame(s1.append(s2)).T)

In [ ]:
def write_log(text):
    with open(log_file, 'a') as lf:
        lf.write(str(text) + '\n')

def get_playlist(conn, pid):
    """
    Query playlists by pid
    :param conn: the Connection object
    :param pid:
    :return: playlist
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM playlists WHERE pid=?", (pid,))
    rows = cur.fetchall()
    playlist = None
    if len(rows) > 0:
        playlist = rows[0]        
    return playlist

def select_track_by_trackuri(conn, track_uri):
    """
    Query tracks by track_uri
    :param conn: the Connection object
    :param track_uri:
    :return: track_id
    """
    cur = conn.cursor()
    cur.execute("SELECT track_id FROM tracks WHERE track_uri=?", (track_uri,))
    rows = cur.fetchall()
    track_id = 0
    if len(rows) > 0:
        track_id = rows[0][0]        
    return track_id

# This function takes somuch time because of loops, so didn't use it
def process_json_data_loop(conn, zip_file, filename, num_playlists, max_track_id):
    print('File: ' + filename)
    write_log('File: ' + filename)

    playlists = []
    ratings = []
    tracks = []
    cnt = 0
    with ZipFile(zip_file) as zipfiles:
        with zipfiles.open(filename) as json_file:
            json_data = json.loads(json_file.read())
            for playlist in json_data['playlists']:
                pl = get_playlist(conn, playlist['pid'])
                if pl != None:
                    write_log('Playlist already exists in database: ' + str(playlist['pid']))
                    print('Playlist already exists in database: ' + str(playlist['pid']))
                    # Playlist already exists in table
                    continue
                else:
                    write_log('Processing Playlist pid: ' + str(playlist['pid']))
                    print('Processing Playlist pid: ' + str(playlist['pid']))
                cnt += 1
                new_playlist = [playlist['pid'], playlist['name'], playlist['num_albums'], playlist['num_artists'], playlist['num_edits'], 
                                playlist['num_tracks'], playlist['collaborative'], playlist['duration_ms'], playlist['modified_at']]
                playlists.append(new_playlist)

                for track in playlist['tracks']:
                    track_uri = track['track_uri'].split(':')[2]
                    write_log('Processing Track: ' + track_uri)
                    # Check if track_uri exists in the tracks list
                    track_id = next((t[0] for t in tracks if track_uri in t), None)
                    if not track_id:
                        write_log('Track not found in tracks list: ' + track_uri)
                        # Check track_uri exists in the database
                        track_id = select_track_by_trackuri(conn, track_uri)
                        if track_id == 0:
                            write_log('Track not found in database: ' + track_uri)
                            album_uri = track['album_uri'].split(':')[2]
                            artist_uri = track['artist_uri'].split(':')[2]
                            # (max_track_id + 1) tracks already exist in database
                            track_id = len(tracks) + (max_track_id + 1)
                            new_track = [track_id, track['track_name'], track_uri, track['album_name'], album_uri, track['artist_name'], artist_uri]
                            write_log('Adding Track to database: ' + str(track_id))
                            tracks.append(new_track)
                    
                    new_rating = [playlist['pid'], track_id, track['pos'], playlist['num_followers']]
                    write_log('Adding Rating for pid:' + str(playlist['pid']) + ' track_id: ' + str(track_id))
                    ratings.append(new_rating)

                if (cnt == num_playlists) and (num_playlists > 0):
                    break

            playlist_cols = ['pid','name','num_albums','num_artists','num_edits','num_tracks','collaborative','duration_ms','modified_at']
            playlists_df = pd.DataFrame(playlists, columns=playlist_cols)
            #print(playlists_df.head())
            write_log('Adding all playlists to database from file: ' + filename)
            playlists_df.to_sql(name='playlists', con=conn, if_exists='append', index=False)

            rating_cols = ['pid', 'track_id', 'pos', 'num_followers']
            ratings_df = pd.DataFrame(ratings, columns=rating_cols)
            #print(ratings_df.head())
            write_log('Adding all ratings to database from file: ' + filename)
            ratings_df.to_sql(name='ratings', con=conn, if_exists='append', index=False)

            track_cols = ['track_id', 'track_name', 'track_uri', 'album_name', 'album_uri', 'artist_name', 'artist_uri']
            tracks_df = pd.DataFrame(tracks, columns=track_cols)
            #print(tracks_df.tail())
            write_log('Adding all tracks to database from file: ' + filename)
            tracks_df.to_sql(name='tracks', con=conn, if_exists='append', index=False)

    if conn:
        conn.close()

In [3]:
import plotly.express as px
from plotly.subplots import make_subplots

with open('../data/read_spotify_mpd_complete_log.txt') as log_file:
   playlist_files = []
   total_tracks = []
   exist_tracks = []
   new_tracks = []
   for line in log_file:
      line = line.strip()
      if 'Created new track_ids' in line:
         new_tracks.append(int(line.split(':')[1].rstrip()))
      if 'Tracks already exist' in line:
         exist_tracks.append(int(line.split(':')[1].rstrip()))
      if 'Total tracks/ratings in this file' in line:
         total_tracks.append(int(line.split(':')[1].rstrip()))
      if 'File: data/mpd.slice' in line:
         playlist_files.append(line.split('.')[-2])

   rows = 300
   exist_tracks_df = pd.DataFrame(zip(playlist_files, exist_tracks), columns=['files', 'num of tracks'])
   exist_tracks_df['tracks'] = 'existing'
   new_tracks_df = pd.DataFrame(zip(playlist_files, new_tracks), columns=['files', 'num of tracks'])
   new_tracks_df['tracks'] = 'new'
   total_tracks_df = pd.DataFrame(zip(playlist_files, total_tracks), columns=['files', 'num of tracks'])
   total_tracks_df['tracks'] = 'total'
   tracks_df = pd.concat([exist_tracks_df.iloc[:rows], total_tracks_df.iloc[:rows], new_tracks_df.iloc[:rows]])
   
   #subfig = make_subplots(rows=2, cols=1, specs=[[{"type": "xy"}], [{"type": "xy"}]])
   #subfig.add_bar(row=1, col=1, y=tracks_df['num of tracks'], x=tracks_df['files'])
   #subfig.add_bar(row=2, col=1, y=new_tracks_df['num of tracks'], x=new_tracks_df['files'])
   fig = px.bar(tracks_df[tracks_df['tracks'] != 'existing'],
             x='files',
             y='num of tracks',
             color='tracks',
             barmode='overlay')
   fig.show()
   
   fig = px.bar(tracks_df[tracks_df['tracks'] != 'total'],
             x='files',
             y='num of tracks',
             color='tracks',
             barmode='stack')
   fig.show()


In [52]:
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())
#json_data = sp.playlist('37i9dQZF1DX9tzt7g58Xlh')
json_data = sp.playlist_items('37i9dQZF1DX9tzt7g58Xlh', offset=0, fields='items.track.artists,items.track.id,items.track.name,items.track.album.id,items.track.album.name,total', additional_types=['track'])
import pprint
print(json_data['total'], len(json_data['items']))
pprint.pprint(json_data['items'])

60 60
[{'track': {'album': {'id': '1bTgKomQYSkKYPD9UI9W4b',
                      'name': 'Enemy (with JID) [from the series Arcane League '
                              'of Legends]'},
            'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/53XhwfbYqKCa1cC15pYq2q'},
                         'href': 'https://api.spotify.com/v1/artists/53XhwfbYqKCa1cC15pYq2q',
                         'id': '53XhwfbYqKCa1cC15pYq2q',
                         'name': 'Imagine Dragons',
                         'type': 'artist',
                         'uri': 'spotify:artist:53XhwfbYqKCa1cC15pYq2q'},
                        {'external_urls': {'spotify': 'https://open.spotify.com/artist/6U3ybJ9UHNKEdsH7ktGBZ7'},
                         'href': 'https://api.spotify.com/v1/artists/6U3ybJ9UHNKEdsH7ktGBZ7',
                         'id': '6U3ybJ9UHNKEdsH7ktGBZ7',
                         'name': 'JID',
                         'type': 'artist',
                         'uri': '

In [47]:
tracks = pd.json_normalize(json_data['tracks']['items'], record_path=['track', 'artists'], meta=[['track', 'name'], ['track', 'id'], ['track', 'album', 'name'], ['track', 'album', 'id']])
print(tracks.head())
tracks.columns

                                                href                      id  \
0  https://api.spotify.com/v1/artists/6Dd3NScHWwn...  6Dd3NScHWwnW6obMFbl1BH   
1  https://api.spotify.com/v1/artists/1pRaG81GsVt...  1pRaG81GsVtaTBuVSpldt2   
2  https://api.spotify.com/v1/artists/3HVdAiMNjYr...  3HVdAiMNjYrQIKlOGxoGh5   
3  https://api.spotify.com/v1/artists/1ZilzPhvZhY...  1ZilzPhvZhYUCSGczNRlyt   
4  https://api.spotify.com/v1/artists/2VYQTNDsvvK...  2VYQTNDsvvKN9wmU5W7xpj   

             name    type                                    uri  \
0            Daya  artist  spotify:artist:6Dd3NScHWwnW6obMFbl1BH   
1        Pop Evil  artist  spotify:artist:1pRaG81GsVtaTBuVSpldt2   
2      Rob Zombie  artist  spotify:artist:3HVdAiMNjYrQIKlOGxoGh5   
3      Eli Sostre  artist  spotify:artist:1ZilzPhvZhYUCSGczNRlyt   
4  Marilyn Manson  artist  spotify:artist:2VYQTNDsvvKN9wmU5W7xpj   

                               external_urls.spotify              track.name  \
0  https://open.spotify.com/ar

Index(['href', 'id', 'name', 'type', 'uri', 'external_urls.spotify',
       'track.name', 'track.id', 'track.album.name', 'track.album.id'],
      dtype='object')

In [61]:
tracks = pd.json_normalize(json_data['items'])
print(len(tracks))
tracks.head()

60


,track.album.id,track.album.name,track.artists,track.id,track.name
0,1bTgKomQYSkKYPD9UI9W4b,Enemy (with JID) [from the series Arcane Leagu...,[{'external_urls': {'spotify': 'https://open.s...,1r9xUipOqoNwggBpENDsvJ,Enemy (with JID) - from the series Arcane Leag...
1,6gblom0Rg1ZWvhMI1w2RUU,Night Visions (Deluxe),[{'external_urls': {'spotify': 'https://open.s...,6nDAs3PRe37fgqLfx51lBB,Radioactive
2,5GlPAy2PRJW06GVFhKwGTl,Evolve,[{'external_urls': {'spotify': 'https://open.s...,0CcQNd8CINkwQfe1RDtGV6,Believer
3,1vAEF8F0HoRFGiYOEeJXHW,Night Visions (Deluxe),[{'external_urls': {'spotify': 'https://open.s...,2Oehrcv4Kov0SuIgWyQY9e,Demons
4,6DdU5f52hpc2gfeLjK374Y,Mercury - Act 1,[{'external_urls': {'spotify': 'https://open.s...,149BxWeweaCqmsyedkNiiE,Wrecked


In [3]:
scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
tracks = sp.current_user_saved_tracks()

In [4]:
tracks['items']

[{'added_at': '2021-09-05T22:32:25Z',
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/53XhwfbYqKCa1cC15pYq2q'},
      'href': 'https://api.spotify.com/v1/artists/53XhwfbYqKCa1cC15pYq2q',
      'id': '53XhwfbYqKCa1cC15pYq2q',
      'name': 'Imagine Dragons',
      'type': 'artist',
      'uri': 'spotify:artist:53XhwfbYqKCa1cC15pYq2q'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BY',
     'BZ',
     'CA',
     'CD',
     'CG',
     'CH',
     'CI',
     'CL',
     'CM',
     'CO',
     'CR',
     'CV',
     'CW',
     'CY',
     'CZ',
     'DE',
     'DJ',
     'DK',
     'DM',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FJ',
    

In [48]:
tracks_df = pd.json_normalize(tracks['items'], record_path=['track', 'artists'], meta=[['added_at'], ['track', 'uri'], ['track', 'name']])
tracks_df = tracks_df.drop_duplicates(subset='track.uri', keep="first")
#[['added_at', 'track.uri','track.name']]
tracks_df['added_at'] = pd.to_datetime(tracks_df['added_at'])
tracks_df = tracks_df.sort_values(by='added_at', ascending=True).set_index('added_at')
tracks_df = tracks_df.last('3M')[['name', 'uri', 'track.uri', 'track.name']]
tracks_df.rename(columns={'track.uri':'uri', 'track.name': 'song', 'name': 'artist', 'uri': 'artist_uri'}, inplace=True)
tracks_df

,artist,artist_uri,uri,song
added_at,,,,
2021-07-27 01:09:07+00:00,Mithoon,spotify:artist:09UmIX92EUH9hAK4bxvHx6,spotify:track:7rMqbvNImA6wy8lxlNUMmj,"Bolo Har Har Har (feat. Mohit Chauhan, Sukhwin..."
2021-08-20 00:51:53+00:00,Karthik,spotify:artist:0LSPREIgGMZXCuKVel7LVD,spotify:track:2OTWTkshnDUT3pKrf0ElOh,Nuvvani Idhi Needani
2021-08-27 20:55:52+00:00,Karunya,spotify:artist:6qrS6jqZfr8HJe3IWkQqv2,spotify:track:59j1ZUoWtcJqiEycTmOl0d,Ola Olaala Ala
2021-09-05 22:32:25+00:00,Imagine Dragons,spotify:artist:53XhwfbYqKCa1cC15pYq2q,spotify:track:1lgN0A2Vki2FTON5PYq42m,Warriors


In [62]:
tracks_df.iloc[2]['uri'] = 'fjdkk'
tracks_df

,artist,artist_uri,uri,song
added_at,,,,
2021-07-27 01:09:07+00:00,Mithoon,spotify:artist:09UmIX92EUH9hAK4bxvHx6,spotify:track:7rMqbvNImA6wy8lxlNUMmj,"Bolo Har Har Har (feat. Mohit Chauhan, Sukhwin..."
2021-08-20 00:51:53+00:00,Karthik,spotify:artist:0LSPREIgGMZXCuKVel7LVD,spotify:track:2OTWTkshnDUT3pKrf0ElOh,Nuvvani Idhi Needani
2021-08-27 20:55:52+00:00,Karunya,spotify:artist:6qrS6jqZfr8HJe3IWkQqv2,fjdkk,Ola Olaala Ala
2021-09-05 22:32:25+00:00,Imagine Dragons,spotify:artist:53XhwfbYqKCa1cC15pYq2q,spotify:track:1lgN0A2Vki2FTON5PYq42m,Warriors


In [50]:
audio_feats = []
all_uris = tracks_df['uri'].tolist()
chunks_uris = [all_uris[i:i + 100] for i in range(0, len(all_uris), 100)]
for chunk in  chunks_uris:
   for attempt in range(10):
         try:
            chunk_audio_feats = sp.audio_features(chunk)
            audio_feats.append(chunk_audio_feats)
         except Exception as e: 
            print(e)
            print('chunk: {}'.format(chunk))
         else:
            break
   else:
         print('Everything failed')

all_audio_feats = [item for sublist in audio_feats for item in sublist if item]

In [51]:
audio_feats_df = pd.DataFrame(all_audio_feats)
audio_feats_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.598,0.813,9,-6.107,1,0.0660,0.2780,0.000000,0.1090,0.311,140.101,audio_features,7rMqbvNImA6wy8lxlNUMmj,spotify:track:7rMqbvNImA6wy8lxlNUMmj,https://api.spotify.com/v1/tracks/7rMqbvNImA6w...,https://api.spotify.com/v1/audio-analysis/7rMq...,294331,4
1,0.688,0.765,9,-4.883,0,0.0624,0.6490,0.000014,0.0712,0.410,100.016,audio_features,2OTWTkshnDUT3pKrf0ElOh,spotify:track:2OTWTkshnDUT3pKrf0ElOh,https://api.spotify.com/v1/tracks/2OTWTkshnDUT...,https://api.spotify.com/v1/audio-analysis/2OTW...,260700,4
2,0.365,0.851,4,-6.238,0,0.0650,0.0964,0.003000,0.2380,0.309,78.068,audio_features,1lgN0A2Vki2FTON5PYq42m,spotify:track:1lgN0A2Vki2FTON5PYq42m,https://api.spotify.com/v1/tracks/1lgN0A2Vki2F...,https://api.spotify.com/v1/audio-analysis/1lgN...,170067,4


In [52]:
fav_songs_feats_df = tracks_df.merge(audio_feats_df, how='right', on="uri")
fav_songs_feats_df

,artist,artist_uri,uri,song,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Mithoon,spotify:artist:09UmIX92EUH9hAK4bxvHx6,spotify:track:7rMqbvNImA6wy8lxlNUMmj,"Bolo Har Har Har (feat. Mohit Chauhan, Sukhwin...",0.598,0.813,9,-6.107,1,0.0660,...,0.000000,0.1090,0.311,140.101,audio_features,7rMqbvNImA6wy8lxlNUMmj,https://api.spotify.com/v1/tracks/7rMqbvNImA6w...,https://api.spotify.com/v1/audio-analysis/7rMq...,294331,4
1,Karthik,spotify:artist:0LSPREIgGMZXCuKVel7LVD,spotify:track:2OTWTkshnDUT3pKrf0ElOh,Nuvvani Idhi Needani,0.688,0.765,9,-4.883,0,0.0624,...,0.000014,0.0712,0.410,100.016,audio_features,2OTWTkshnDUT3pKrf0ElOh,https://api.spotify.com/v1/tracks/2OTWTkshnDUT...,https://api.spotify.com/v1/audio-analysis/2OTW...,260700,4
2,Imagine Dragons,spotify:artist:53XhwfbYqKCa1cC15pYq2q,spotify:track:1lgN0A2Vki2FTON5PYq42m,Warriors,0.365,0.851,4,-6.238,0,0.0650,...,0.003000,0.2380,0.309,78.068,audio_features,1lgN0A2Vki2FTON5PYq42m,https://api.spotify.com/v1/tracks/1lgN0A2Vki2F...,https://api.spotify.com/v1/audio-analysis/1lgN...,170067,4
